In [1]:
import tweepy
import time
import sys
import csv
from datetime import datetime, timedelta

# latitude = -7.059035     # geographical centre of search
# longitude = 110.443972   # geographical centre of search
latitude = -0.885296
longitude = 119.863314
max_range = 110          # search range in kilometres
days_before = 11
outfile = "pantai_talise_tweets.csv"

consumer_key = 'JH8yplw1cyy6VCuu7bZTIaaMG'
consumer_secret = 'HwKOAbZLUUPIHjcS8ulK2mNyJRmeHSXmK9bQUuyUnT9CSAYJc9'
access_token = '477563933-f6iCy9TdX9CgJanbyz37qzYXOgWTPmpJEYIqMjRS'
access_secret = 'LvhDkE7BB8y09wPaZhT8qhzIShQl7OgDUrA5gJggVHOen'

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [3]:
csvfile = open(outfile, "w", newline='', encoding='utf-8')
csvwriter = csv.writer(csvfile)

row = [ "id_user", "username", "created_at", "latitude", "longitude", "text" ]
csvwriter.writerow(row)

53

In [4]:
travel_query = "wisata OR liburan OR piknik OR travel OR #travel OR #liburan OR #wisata OR #explore OR #vacation OR #traveler"
culinary_query = "kuliner OR makan OR minum OR #kuliner OR #food OR #foodies OR #culinary OR warung OR restoran"

coba_query = '"Kelok 9" -filter:retweet'

search_query = travel_query + " -filter:retweet"

In [5]:
# terbatas hanya untuk 23 query
# terbatas 
coba_query = '"lawang sewu" OR "kota lama" OR "tbrs" OR "maerakaca" OR "pantai marina" OR \
              "simpang lima" OR "curug lawe" OR "sidomukti" OR "eling bening" OR "prpp" OR \
              "kopeng" OR "ronggowarsito" OR "palagan ambarawa" OR "bukit cinta" OR "tugu muda" OR \
              "sampokong" OR "tanjung emas" OR "pondok kopi" OR "bantir" OR "banyubiru" OR \
              "bandungan" OR "watugong" OR "getasan" \
              -filter:retweet'
pantai_query = '"pantai marina" OR "pantai maron" OR ""'
pantai_talise_query = 'pantai talise'

In [6]:
%%time

result_count = 0
now = datetime.today()
date_before = now - timedelta(days=days_before)

c = tweepy.Cursor(api.search,
                  q=pantai_talise_query,
                  tweet_mode='extended',
                  count=100,
                  show_user=True,
                  geocode = "%f,%f,%dkm" % (latitude, longitude, max_range)).items()
# c = tweepy.Cursor(api.search,
#                   q=pantai_talise_query,
#                   tweet_mode='extended',
#                   show_user=True,
#                   count=100).items()

while True:
    try:
        tweet = c.next()
        if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
            # get tweets maximum before x days
#             if tweet.created_at < date_before:
#                 break
            
            if tweet.geo:
                csvwriter.writerow([tweet.user.id, tweet.user.screen_name, tweet.created_at, 
                                   tweet.geo['coordinates'][0], tweet.geo['coordinates'][1], tweet.full_text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')])
                result_count += 1
                print ("got %d results" % result_count)
            else:
                csvwriter.writerow([tweet.user.id, tweet.user.screen_name, tweet.created_at, 
                                    None, None, tweet.full_text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')])
                result_count += 1
                print ("got %d results" % result_count)

    except tweepy.TweepError as e:
        print(e)
        print("sleeping")
        time.sleep(15 * 60)
        continue
    except StopIteration:
        break
csvfile.close()

got 1 results
got 2 results
got 3 results
got 4 results
got 5 results
got 6 results
got 7 results
got 8 results
got 9 results
got 10 results
got 11 results
got 12 results
got 13 results
got 14 results
Wall time: 1.21 s


In [7]:
from bs4 import BeautifulSoup
import requests

url = "https://id.wikipedia.org/wiki/Daftar_tempat_wisata_di_Indonesia"
source = requests.get(url)
plain_text = source.text

soup = BeautifulSoup(plain_text, "html.parser")

f = open("list_wisata.txt", "w")

for li in soup.findAll('li'):
    a = li.find("a", title=True)
    
    if a:
        f.write(a.text + "\n")
    else:
        f.write(li.text + "\n")
f.close()

UnicodeEncodeError: 'charmap' codec can't encode character '\ufffc' in position 65: character maps to <undefined>

In [ ]:
with open('list_wisata.txt', 'r') as file:
    list_wisata = [repr(line.rstrip()) for line in file]
join_lw = ' OR '.join(list_wisata)